# Rover Project Test Notebook

### <span style="color:blue"> Table of Content: </span>

<a href='#1'>1. Project Notes </a>

<a href='#2'>2. Import required Python Libraries </a>

<a href='#3'>3. Prespective Transformation </a>

<a href='#4'>4. Color Thresholding </a>

<a href='#5'>5. Thresholding of Warped Images </a>

<a href='#6'>6. Coordinates Transformation </a>

<a href='#7'>7. Create Data Container Class </a>

<a href='#8'>8. Process_Image Function </a>

<a href='#9'>9. Make a video from processed image data </a>

<a href='#10'>10. Play Video </a>


## 1.  <span style="color:blue"> Project Notes: </span><a id='1'></a>

* Simulator Selected Screen Resolution: 1024 x 640
* Simulator Graphics Quality: Good

## 2. <span style="color:blue"> Import required Python Libraries </span><a id='2'></a>

In [ ]:
%%HTML
<style> code {background-color : orange !important;} </style>

In [ ]:
%matplotlib inline

# required imports
import numpy as np                       # For array maths
import matplotlib.pyplot as plt          # For ploting results
import matplotlib.image as mpimg         # For images reading
import cv2                               # OpenCV for perspective transform
import glob                              # For reading in a list of images from a folder
import pandas as pd                      # For reading csv file as a dataframe
import imageio                           # For reading image and video.
imageio.plugins.ffmpeg.download()

## 3. <span style="color:blue"> Prespective Transformation </span><a id='3'></a>

In [ ]:
# Read Calibration Data and show sample

example_grid = '../calibration_images/example_grid1.jpg'
example_rock = '../calibration_images/example_rock1.jpg'

grid_img = mpimg.imread(example_grid)       # get grid example
rock_img = mpimg.imread(example_rock)       # get rock example


# define prespecive transformation function

def perspect_transform(img, src, dst):

    M = cv2.getPerspectiveTransform(src, dst)  # Get transform matrix using src and dst boxes
    warped = cv2.warpPerspective(img,          # Warp image keep same size as input image 
                                 M, 
                                 (img.shape[1], img.shape[0]))
    
    
    return warped

# Defined to warp the image to a grid where each 10x10 pixel square represents 1 square meter

hs_size = 10/2             # half the size of one square
b_ofst  = 6                # bottom offset to account for distance from rover edge to 1st camera visiable point
x_cntr  = grid_img.shape[1]/2 # center of the image x axis
y_end   = grid_img.shape[0]   # end of y or bottom of image

src = np.float32([
                 [14, 140],      # Left Bottom
                 [301 ,140],     # Right Bottom
                 [200, 96],      # Right Top
                 [118, 96]       # Left Top
                 ])              # above data captured from grid calibration image.

dst = np.float32([
                 [x_cntr - hs_size, y_end - b_ofst],             # Left Bottom
                 [x_cntr + hs_size, y_end - b_ofst],             # Right Bottom
                 [x_cntr + hs_size, y_end - 2*hs_size - b_ofst], # Right Top
                 [x_cntr - hs_size, y_end - 2*hs_size - b_ofst], # Left Top
                 ])

# warp images using defined src and dst boxes

navi_w = perspect_transform(grid_img, src, dst) #warp the navi/obst image
rock_w = perspect_transform(rock_img, src, dst) #warp the rock image


# Display images
f, ax = plt.subplots(2, 2, figsize=(24,12))
f.suptitle('Prespective Transformation', fontsize=28)

ax[0,0].imshow(grid_img)
ax[0,0].set_title('Grid Sample', fontsize=24)

ax[1,0].imshow(rock_img)
ax[1,0].set_title('Rock Sample', fontsize=24)

ax[0,1].imshow(navi_w)
ax[0,1].set_title('Grid Warped', fontsize=24)

ax[1,1].imshow(rock_w)
ax[1,1].set_title('Rock Warped', fontsize=24)

plt.subplots_adjust(wspace=0.1)
plt.show()

# Save figure for project report
f.savefig('../output/warp_fun.jpg')

## 4. <span style="color:blue"> Color Thresholding </span><a id='4'></a>

In [ ]:
# read all images from test data set folder
path = '../test_dataset/IMG/*'                # path to recorded images
img_list = glob.glob(path)                    # read all images to a glob object

idx = np.random.randint(0, len(img_list)-1)   # Pick on random image
sample_img = mpimg.imread(img_list[idx])


# Define a function to threshold warped image and show navigable/obstacles areas
def navi_thresh(img):

    # Threshold of RGB > 160 does a nice job of identifying ground pixels only
    rgb_thr=(160,160,160)
     
    # mask will contain a boolean array with "True" for each pixel above threshold
    mask = (img[:,:,0] > rgb_thr[0]) \
         & (img[:,:,1] > rgb_thr[1]) \
         & (img[:,:,2] > rgb_thr[2])

    # Create an array of zeros same xy size as img, but single channel
    # Index the array with the mask and set to 1
    navi = np.zeros_like(img[:,:,0])
    navi[mask] = 1
    
    navi[:int(img.shape[0]*.5),:]=0 # clip upper 50% of image to improve fidelity
    
    # Create an array of ones same xy size as img, but single channel
    # Index the array with the mask and set to 0
    obst = np.ones_like(img[:,:,0])
    obst[mask] = 0
    
    obst[:int(img.shape[0]*.5),:]=1 # clip upper 50% of image to improve fidelity
    
    return navi,obst                        # Return both images


# Define a function to threshold rock calibration image and isolate the rock.
def rock_thresh(img):
    
    hsv_img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    lower = np.array([0,200,100])
    upper = np.array([179,255,255])
    
    rock_t = cv2.inRange(hsv_img, lower, upper)
      
    return rock_t


# test thresholding functions on sample images
navi_t,obst_t = navi_thresh(sample_img)  # Threshold warped image to show both navigable and obstcales areas
rock_t        = rock_thresh(rock_img)           # Threshold calibration image to isolate the rock

# Display the three threshold images

f, ax = plt.subplots(3, 2, figsize=(20,15))
f.suptitle('Color Thresholding', fontsize=28)

ax[0,0].imshow(sample_img)
ax[0,0].set_title('Sample Image', fontsize=24)

ax[0,1].imshow(navi_t,cmap='gray')
ax[0,1].set_title('Navigable Area Threshold', fontsize=24)

ax[1,0].imshow(sample_img)
ax[1,0].set_title('Sample Image', fontsize=24)

ax[1,1].imshow(obst_t,cmap='gray')
ax[1,1].set_title('Obstacles Area Threshold', fontsize=24)

ax[2,0].imshow(rock_img)
ax[2,0].set_title('Rock Image', fontsize=24)

ax[2,1].imshow(rock_t,cmap='gray')
ax[2,1].set_title('Rock Threshold', fontsize=24)

plt.show()

# Save figure for project report
f.savefig('../output/thresh_fun.jpg')


## 5. <span style="color:blue"> Thresholding of Warped Images </span><a id='5'></a>

In [ ]:
# test thresholding functions on wraped images

navi_wt,obst_wt = navi_thresh(navi_w)  # Threshold warped image to show both navigable and obstcales areas
rock_wt         = rock_thresh(rock_w)  # Threshold calibration image to isolate the rock

# Display the three threshold images

f, ax = plt.subplots(3, 2, figsize=(20,15))
f.suptitle('Thresholding of Warped Images', fontsize=28)

ax[0,0].imshow(sample_img)
ax[0,0].set_title('Sample Image', fontsize=24)

ax[0,1].imshow(navi_wt,cmap='gray')
ax[0,1].set_title('Navigable Area Thres/Warp', fontsize=24)

ax[1,0].imshow(sample_img)
ax[1,0].set_title('Sample Image', fontsize=24)

ax[1,1].imshow(obst_wt,cmap='gray')
ax[1,1].set_title('Obstacles Area Thres/Warp', fontsize=24)

ax[2,0].imshow(rock_img)
ax[2,0].set_title('Rock Image', fontsize=24)

ax[2,1].imshow(rock_wt,cmap='gray')
ax[2,1].set_title('Rock Thres/Warp', fontsize=24)

plt.subplots_adjust(wspace=0.1)
plt.show()

# Save figure for project report
f.savefig('../output/threshwarp_fun.jpg')


## 6. <span style="color:blue"> Coordinates Transformation </span><a id='6'></a>
### 6.1 Coordinate Transformations Functions

In [ ]:
# Define a function to convert from image coords to rover coords
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[1]/2 ).astype(np.float)
    return x_pixel, y_pixel

# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

# Define a function to map rover space pixels to world space
def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated


# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

### 6.2 Test Transformations Functions: rover_coords() and to_polar_coords()

In [ ]:
# Grab another random image
idx = np.random.randint(0, len(img_list)-1)
navi_img = mpimg.imread(img_list[idx])

# Transform images
navi_w = perspect_transform(navi_img, src, dst)
navi_t,obst_t = navi_thresh(navi_w)

# Calculate pixel values in rover-centric coords and distance/angle to all pixels
xpix, ypix = rover_coords(navi_t)
dist, angles = to_polar_coords(xpix, ypix)
mean_dir = np.mean(angles)
angle = np.clip(mean_dir*180/np.pi,-15,15)

print("dist =",dist[0:3])
print("angles =",angles[0:3])
print("mean_dir =",mean_dir)
print("Angle =",angle)




# Show results
f,ax =  plt.subplots(2, 2, figsize=(24,12))
f.suptitle('Rover-centric coordinates', fontsize=28)

ax[0,0].set_title('Sample Image', fontsize=24)
ax[0,0].imshow(navi_img)

ax[0,1].set_title('Warped', fontsize=24)
ax[0,1].imshow(navi_w)

ax[1,0].set_title('Warped and Thresholded', fontsize=24)
ax[1,0].imshow(navi_t, cmap='gray')

ax[1,1].set_title('Rover-centric with angle', fontsize=24)
ax[1,1].plot(xpix, ypix, '.')

ax[1,1].set_xlim([0, 160])
ax[1,1].set_ylim([-160, 160])

arrow_length = 100
x_arrow = arrow_length * np.cos(mean_dir)
y_arrow = arrow_length * np.sin(mean_dir)

ax[1,1].arrow(0, 0, x_arrow, y_arrow, color='red', zorder=2, head_width=10, width=1)

# Save figure for project report
f.savefig('../output/rover_coords.jpg')

### 6.3. Test Transformations Functions: rotate_pix(), translate_pix(), and pix_to_world()

In [ ]:
# Rover yaw values will come as floats from 0 to 360
# Generate a random value in this range
# Note: you need to convert this to radians before adding to pixel_angles
rover_yaw = np.random.random(1)*360

# Generate a random rover position in world coords
# Position values will range from 20 to 180 to avoid the edges in a 200 x 200 pixel world
rover_xpos = np.random.random(1)*160 + 20
rover_ypos = np.random.random(1)*160 + 20

worldmap = np.zeros((200, 200))  # create empty image for use in testing
scale = 10                       # world map scale

# Note: Since we've chosen random numbers for yaw and position, 
# multiple run of the code will result in different outputs each time.

# Get navigable pixel positions in world coords

x_world, y_world = pix_to_world(xpix, ypix, 
                                rover_xpos, rover_ypos, rover_yaw, 
                                worldmap.shape[0], 
                                scale)


# Add pixel positions to worldmap

worldmap[y_world, x_world] += 1

# Show results

print('Xpos =', rover_xpos, 'Ypos =', rover_ypos, 'Yaw =', rover_yaw)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
f.suptitle('World Map Coordinates', fontsize=28)

#f.tight_layout()

ax1.plot(xpix, ypix, '.')
ax1.set_title('Rover Space', fontsize=24)
ax1.set_ylim(-160, 160)
ax1.set_xlim(0, 160)

ax2.imshow(worldmap, cmap='gray')
ax2.set_title('World Space', fontsize=24)
ax2.set_ylim(0, 200)
ax2.set_xlim(0, 200)

plt.show()

# Save figure for project report
f.savefig('../output/world_space.jpg')

## 7. <span style="color:blue"> Create Data Container Class</span><a id='7'></a>

### 7.1 Read CSV file into a dataframe

CSV file generated from the Rover Simulator Recorder will be having the following data columns:

(Path, SteerAngle, Throttle, Brake, Speed, X_Position, Y_Position, Pitch, Yaw, Roll)

Number of rows will depend on how long the recording was done. for each row there will be on captured image in /IMG subfolder.

You can pick one of the following datasets:
#### rec1_dataset
#### rec2_dataset
#### test_dataset

In [ ]:
csv_path = '../rec2_dataset/robot_log.csv'            # path to CSV file

df = pd.read_csv(csv_path,delimiter=';',decimal='.')  # read all columns/rows into dataframe

print(df[0:3])                                        # show 1st 3 samples

### 7.3 Create a list of images pathnames

In [ ]:
csv_img_list = df["Path"].tolist()  # Create list of image pathnames

print(csv_img_list[0])              # Show 1st sample

### 7.2 Read Ground Truth Map and create a 3-Channel image from it

In [ ]:
map_path = '../calibration_images/map_bw.png'     # path to ground_truth image file

ground_truth = mpimg.imread(map_path)             # read ground_truth image

ground_truth_3d = np.dstack((ground_truth,      # Blue channel
                             ground_truth*255,  # Green channel
                             ground_truth       # Red channel
                            )
                           ).astype(np.float)   # all BGR numbers are float

# show map image

f,ax =  plt.subplots(1, 1, figsize=(6,6))

ax.set_title('Ground Truth Map', fontsize=24)
ax.set_ylim(0,200)
ax.set_xlim(0,200)
ax.imshow(ground_truth_3d.astype(np.uint8))

# Save figure for project report
f.savefig('../output/ground_truth.jpg')

### 7.3 Create Data Container Class

 Will create a data container class and populate it with saved data from csv file and Woldmap. Worldmap is instantiated as 200 x 200 grid corresponding to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels). This encompasses the full range of output position values in x and y from the sim.

In [ ]:
# define the class

class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw  = df["Yaw"].values
        self.pitch= df["Pitch"].values
        self.roll = df["Roll"].values
        self.sangl= df["SteerAngle"].values
        self.brake= df["Brake"].values
        self.throt= df["Throttle"].values
        self.speed= df["Speed"].values
        self.count = 0 # This will be a running index
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

        
# Instantiate a Databucket()

data = Databucket()

# this will be a global variable/object that you can refer to in the process_image() function.

## 8. <span style="color:blue"> Process_Image Function </span><a id='8'></a>

In [ ]:
# Define a function to pass stored images to reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video

def process_image(img):

# 1) Define source and destination points for perspective transform
#------------------------------------------------------------------
    
    scale    = 10             # each 10x10 pixel square represents 1 square meter
    hs_size  = scale/2        # half the size of one square
    b_ofst   = 6              # bottom offset to account for distance from rover edge to 1st camera visiable point
    x_cntr   = img.shape[1]/2 # center of the image x axis
    x_max    = img.shape[1]   # end of x or right edge of image
    y_max    = img.shape[0]   # end of y or bottom edge of image
    
    rvr_xpos = data.xpos[data.count]
    rvr_ypos = data.ypos[data.count]
    rvr_yaw  = data.yaw[data.count]
    wrl_shp0 = data.worldmap.shape[0]
    
    
    src = np.float32([
                     [14, 140],      # Left Bottom
                     [301 ,140],     # Right Bottom
                     [200, 96],      # Right Top
                     [118, 96]       # Left Top
                     ])              # above data captured from grid calibration image.

    dst = np.float32([
                     [x_cntr - hs_size, y_max - b_ofst],             # Left Bottom
                     [x_cntr + hs_size, y_max - b_ofst],             # Right Bottom
                     [x_cntr + hs_size, y_max - 2*hs_size - b_ofst], # Right Top
                     [x_cntr - hs_size, y_max - 2*hs_size - b_ofst], # Left Top
                     ])
    
# 2) Apply perspective transform
#------------------------------------------------------------------

    img_w = perspect_transform(img, src, dst) #warp the image

# 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
#------------------------------------------------------------------
    navi_wt,obst_wt = navi_thresh(img_w)  # Threshold warped image to show both navigable and obstcales areas
    rock_wt = rock_thresh(img_w)           # Threshold calibration image to isolate the rock
    
# 4) Convert thresholded image pixel values to rover-centric coords
#------------------------------------------------------------------
    navix_pix, naviy_pix = rover_coords(navi_wt)  # convert navigable area thresholded to rover coords.
    obstx_pix, obsty_pix = rover_coords(obst_wt)  # convert obstacle area thresholded to rover coords.
    rockx_pix, rocky_pix = rover_coords(rock_wt)  # convert rock thresholded to rover coords.
        
# 5) Convert rover-centric pixel values to world coords
#------------------------------------------------------------------
    
    naviy_wld, navix_wld = pix_to_world(navix_pix, naviy_pix,          # convert navigable area
                                        rvr_xpos, rvr_ypos, rvr_yaw,
                                        wrl_shp0,scale)
    
    obsty_wld, obstx_wld = pix_to_world(obstx_pix, obsty_pix,          # convert obsticale area
                                        rvr_xpos, rvr_ypos, rvr_yaw,
                                        wrl_shp0,scale)
    
    rocky_wld, rockx_wld = pix_to_world(rockx_pix, rocky_pix,          # convert rock area
                                        rvr_xpos, rvr_ypos, rvr_yaw,
                                        wrl_shp0,scale)
    
    
    
# 6) Update worldmap (to be displayed on right side of screen)
#------------------------------------------------------------------
    data.worldmap[obstx_wld, obsty_wld, 0] = 255 # set obsticale area pixels to RED
    data.worldmap[navix_wld, naviy_wld, 2] = 255 # set navigable area pixels to BLUE
    data.worldmap[rockx_wld, rocky_wld, :] = 255 # set rock area to WHITE

    navi = data.worldmap[:,:,2]==255 # index of all navigable pixels
    data.worldmap[navi,0]=0          # reset corresponding obsticale pixels
    
    
# 7) Make a mosaic image
#------------------------------------------------------------------     
    frame = np.zeros((img.shape[0] + data.worldmap.shape[0], 
                      img.shape[1]*2, 
                      3))                          # create a blank image
    frame[0:img.shape[0], 0:img.shape[1]] = img    # add camera image to top-left
    frame[0:img.shape[0], img.shape[1]:]  = img_w  # add warped image to top-right

    map_add = cv2.addWeighted(data.worldmap,1,     # mix world map with ground truth
                              data.ground_truth,0.5,
                              0) 

    
# 8) Add rover location and direction markers to mosaic image
#------------------------------------------------------------------    

    
    ryaw  = rvr_yaw*np.pi/180               # convert yaw from deg to rad
    
    ac = (0,0,255)                          # arrow color
    al = 5                                  # arrow half length
    fx = int(rvr_xpos+(al * np.cos(ryaw)))  # arrow head point x based on current rover yaw
    fy = int(rvr_ypos+(al * np.sin(ryaw)))  # arrow head point y based on current rover yaw
    bx = int(rvr_xpos)                      # arrow start point x
    by = int(rvr_ypos)                      # arrow start point y
            
    cv2.arrowedLine(map_add,(bx,by),(fx,fy),ac)
        
    frame[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)  # add mixed map to bottom-left


# 9) Show telemetry values (bottom-right)
#------------------------------------------------------------------

    # Calculate some statistics on the map results
    
    # total pix in the navigable area
    tot_nav_pix = np.float(len((data.worldmap[:,:,2].nonzero()[0])))  
    
    # how many correspond to truth
    good_nav_pix = np.float(len(((data.worldmap[:,:,2] > 0) & (data.ground_truth[:,:,1] > 0)).nonzero()[0]))
    
    # how many do not correspond to truth
    bad_nav_pix = np.float(len(((data.worldmap[:,:,2] > 0) & (data.ground_truth[:,:,1] == 0)).nonzero()[0]))
    
    # Grab the total number of map pixels
    tot_map_pix = np.float(len((data.ground_truth[:,:,1].nonzero()[0])))
    
    # Calculate the percentage of ground truth map that has been successfully found
    perc_mapped = round(100*good_nav_pix/tot_map_pix, 1)
    
    # number of good map pixel detections divided by total pixels found to be navigable terrain
    if tot_nav_pix > 0:
        fidelity = round(100*good_nav_pix/(tot_nav_pix), 1)
    else:
        fidelity = 0



    # text style
    tox =img.shape[1]+20        # Text X offset from top left corner
    toy =img.shape[0]+20        # Text Y offset from top left corner
    tc  =(255, 255, 255)          # Text color
    tf  =cv2.FONT_HERSHEY_COMPLEX # Text font
    ts  =0.4                      # Text size
    
    # text value
    txt1="Xpos="+str(np.round(data.xpos[data.count],2))+\
         " Ypos="+str(np.round(data.ypos[data.count],2))
        
    txt2="Yaw="+str(np.round(data.yaw[data.count],2))+\
         " Roll="+str(np.round(data.roll[data.count],2))
            
    txt3="Pitch="+str(np.round(data.pitch[data.count],2))+\
         " S.Angle="+str(np.round(data.sangl[data.count],2))
         
    
    txt4="Throttle="+str(np.round(data.throt[data.count],2))+\
         " Speed="+str(np.round(data.speed[data.count],2))
         
    txt5="Brake="+str(np.round(data.brake[data.count],2))
    
    txt6="Mapped: "+str(perc_mapped)+'%'
    
    txt7="Fidelity: "+str(fidelity)+'%'
    
    # Add text to frame
    cv2.putText(frame,txt1,(tox,toy+20),tf,ts,tc,1)
    cv2.putText(frame,txt2,(tox,toy+40),tf,ts,tc,1)
    cv2.putText(frame,txt3,(tox,toy+60),tf,ts,tc,1)
    cv2.putText(frame,txt4,(tox,toy+80),tf,ts,tc,1)
    cv2.putText(frame,txt5,(tox,toy+100),tf,ts,tc,1)
    cv2.putText(frame,txt6,(tox,toy+120),tf,ts,tc,1)
    cv2.putText(frame,txt7,(tox,toy+140),tf,ts,tc,1)
#------------------------------------------------------------------
    
    if data.count < len(data.images) - 1:
        data.count += 1 # Keep track of the index in the Databucket()
    
    return frame

In [ ]:
# test function with one sample
data = Databucket()

# Grab random image from data
idx = np.random.randint(0, len(data.images) - 1) 
navi_img = mpimg.imread(data.images[idx])
tst_frame = process_image(navi_img)

# Show frame
f,ax =  plt.subplots(1, 1, figsize=(14,14))
ax.set_title('One Sample Video Frame', fontsize=20)
ax.imshow(tst_frame.astype(np.uint8))

# Save figure for project report
f.savefig('../output/test_frame.jpg')

## 9. <span style="color:blue"> Make a video from processed image data </span><a id='9'></a>
  

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=50) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

## 10. <span style="color:blue"> Play Video </span><a id='10'></a>
### 10.1. Video Player using HTML

This next cell should function as an inline video player
If this fails to render the video, try running the following cell (alternative video rendering method). You can also simply have a look at the saved mp4 in your /output folder

In [ ]:
output = '../output/test_mapping.mp4'
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### 10.2. Alternative video rendering method

In [ ]:
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))